<a href="https://colab.research.google.com/github/FabiolaAJ/chatbot-assistente-de-vagas/blob/main/Chatbot_Assistente_de_Vagas_de_emprego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import os
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
from IPython.display import clear_output

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [162]:
##########################################
# --- Agente 1: Buscador de Vagas --- #
##########################################
def agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente simpático especializado na busca de vagas de emprego.
        Use a ferramenta de busca do google (google_search) para encontrar até 5 vagas relevantes e atuais para a profissão e senioridade do usuário.

          1. **Interpretação:** Analise profissão e senioridade. Se a senioridade faltar, pergunte.
          2. **Busca (google_search):** Combine profissão, área, senioridade e informe se a vaga é hibrida, presencial ou remoto. Foque em resultados que provavelmente contenham links diretos para a página da vaga.
          3. **Filtragem:** Priorize vagas recentes verificando a data de hoje (data_de_hoje). Descarte vagas antigas ou encerradas.
          4. **Resposta (máximo 5 vagas):** Para cada vaga, retorne título, empresa, localização (se disponível), breve descrição com requisitos para vaga, não faça questionamentos adicionais para o usuário.
          5. **Link:** [**Link direto para a página da vaga**. Este é um campo obrigatório. Tente ao máximo encontrar o link nos resultados da busca.]
          6. **Sem Resultados:** Se não houver vagas relevantes e atuais, informe o usuário.

          Priorize informações precisas e atuais.

        """,
        description="Agente que busca informações de vagas de empregos no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Profissão: {profissao} \n Area:{area} \n Senioridade:{senioridade} \n Data de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [145]:
################################################
# --- Agente 2: Planejador de carreira e recrutamento --- #
################################################
def agente_planejador_de_carreira(profissao, area, senioridade, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em carreira e recrutamento muito gentil e simpático, com vasta experiência em ajudar candidatos a se prepararem para processos seletivos.
        Gere dicas para se destacar, considerando os seguintes tópicos:
             1. Quais habilidades e experiências o candidato deve enfatizar?
             2. Forneça até 2 dicas concisas sobre como o usuário pode se organizar para se candidatar as vagas (ex: adaptar currículo, pesquisar a empresa).
             3. Ao final cite dicas gerais de como se destacar no mercado de trabalho de acordo com a profissão do usuário
        """,
        description="Agente planejador especialista em carreira",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Profissão: {profissao} \n Area:{area}  \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [154]:
def obter_input_obrigatorio(mensagem_inicial, mensagem_erro):
    # Exibe a mensagem inicial para o usuário, solicitando a entrada.
    valor = input(f"\n{mensagem_inicial} ")

    # Inicia um loop 'while' que continuará executando enquanto a condição for verdadeira.
    # A condição 'not valor.strip()' verifica se a variável 'valor' está vazia ou contém apenas espaços em branco.
    # 'valor.strip()' remove os espaços em branco do início e do final da string.
    # Se após a remoção dos espaços, a string estiver vazia, a condição será 'True' e o loop continuará.
    while not valor.strip():
        # Exibe a mensagem de erro informando ao usuário que a entrada é obrigatória.
        print(f"{mensagem_erro}")

        # Solicita novamente a entrada do usuário, exibindo a mensagem inicial novamente.
        valor = input(f"\n{mensagem_inicial}")

    # Quando o loop 'while' terminar (ou seja, o usuário digitou algo que não é apenas espaço),
    # a função retorna o valor digitado, com quaisquer espaços em branco iniciais e finais removidos
    # usando o método 'strip()'.
    return valor.strip()

In [152]:
def exibir_resultados(profissao, area, senioridade):
    print(f"")

    data_de_hoje = date.today().strftime("%d/%m/%Y")

    resposta_buscador = agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje)
    print("\n 📝 Essas foram as vagas encontradas! \n")
    display(to_markdown(resposta_buscador))
    print("--------------------------------------------------------------")

    print("\n🔄 Processando resultados de planejamento...")
    resposta_planejador = agente_planejador_de_carreira(profissao, area, senioridade, resposta_buscador)
    print("\n 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 \n")
    display(to_markdown(resposta_planejador))
    print("--------------------------------------------------------------")

    return

In [165]:
def main():
  exibir_questionario = "1";

  while exibir_questionario == "1" :
    clear_output(wait=True)

    print("\n Olá, tudo bem por aí? 😊 Que bom te ter aqui!")
    print("\n Sou seu assistente virtual focado , pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te deixar fera para as entrevistas! 🎉")
    print("\n--------------------------------------------------------------")

    profissao = obter_input_obrigatorio("❓Me conta qual a profissão que te interessa encontrar vagas hoje? ", "/n Hmm, para que eu te ajude a achar as melhores vagas, preciso que me conte qual profissão você busca, tá bem? 😉 ")
    area = obter_input_obrigatorio("❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend) " , "É importante me informar qual área específica dentro dessa profissão deseja atuar!😊 ")
    senioridade = obter_input_obrigatorio("❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...) ", "Porfavor, digite seu nível de experiência para refinar a busca! 😊 ")

    print("\n🔄 Processando resultados...")
    exibir_resultados(profissao, area, senioridade)

    print("Se deseja procurar outra vaga, digite 1: \n")
    exibir_questionario = input("")

    if(exibir_questionario != '1'):
      print("\n Até a próxima! 😊")
      break;


main()




 Olá, tudo bem por aí? 😊 Que bom te ter aqui!

 Sou seu assistente virtual focado , pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te deixar fera para as entrevistas! 🎉

--------------------------------------------------------------

❓Me conta qual a profissão que te interessa encontrar vagas hoje?  design

❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend)  ux

❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...)  pleno

🔄 Processando resultados...


 📝 Essas foram as vagas encontradas! 



> Ok, vamos procurar vagas de emprego para a área de UX Design para nível pleno.
> 
> 
> Com certeza! Aqui estão algumas vagas de UX Design de nível pleno que encontrei:
> 
> 1.  **UX Designer (Pleno/Sênior) na Leega Consultoria**
> 
>     *   **Localização:** Remoto
>     *   **Requisitos:** Mínimo de 4 anos de experiência na área; experiência no segmento financeiro (desejável); pensamento analítico, bom relacionamento interpessoal, criatividade, capacidade de trabalhar com metodologias ágeis, pós-graduação em UX, Design Research e conhecimento básico de Lógica e Linguagem de Programação.
>     *   **Link:** [https://www.gupy.us/vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEcZdr0ri-IAGhISi8Mdc3dxlWjl1Ob\_YHuCOH1KtpBZQqE-cI0Ee2OvgWzs\_l\_p1DUp5sGyN0Ftv8PliwrSlRIyy2ZkhJEUDnrx00WTAI7emyyH9yVUbQgqXKVHxwLpud0OY0NkhyJMOMnwQ0gW5GDzBSzbRq45\_NLIQisY0qKAmbH\_Ep4Sx3IkbcDa7towf\_eUFyAVsUAZIcfT4I=](https://www.gupy.us/vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEcZdr0ri-IAGhISi8Mdc3dxlWjl1Ob_YHuCOH1KtpBZQqE-cI0Ee2OvgWzs_l_p1DUp5sGyN0Ftv8PliwrSlRIyy2ZkhJEUDnrx00WTAI7emyyH9yVUbQgqXKVHxwLpud0OY0NkhyJMOMnwQ0gW5GDzBSzbRq45_NLIQisY0qKAmbH_Ep4Sx3IkbcDa7towf_eUFyAVsUAZIcfT4I=)
> 2.  **UX Designer - Pleno no Grupo Regazzo**
> 
>     *   **Localização:** Remoto
>     *   **Requisitos:** Mínimo de 3 anos de experiência comprovada como UX Designer; domínio de Figma; conhecimento em Design Centrado no Usuário; capacidade de transformar requisitos de negócio em soluções funcionais; excelente comunicação e facilidade para trabalhar com times multidisciplinares.
>     *   **Link:** [https://www.recrutei.com.br/vaga/ux-designer-pleno-remoto](https://www.recrutei.com.br/vaga/ux-designer-pleno-remoto)
> 3.  **Product UI/UX Designer Pleno na Trade Technology (SIVEE)**
> 
>     *   **Localização:** 100% Remoto
>     *   **Requisitos:** Experiência comprovada na criação, documentação e handoff de produtos digitais; sólida vivência como UI Designer ou UX/UI Designer; domínio da ferramenta Figma; experiência na documentação de estilos, padrões e comportamentos de interface; capacidade analítica e foco em melhoria contínua da experiência do usuário.
>     *   **Link:** [https://www.gupy.us/vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGQZiNg6ZP2RNUjze8NqOi5OIR2j0Mu4ZzuUPwqusPKCNFN2B6TGA7ltzYPOuZKRdFq7Jx8N5WY-wi58Z7-ZZsd3yeN-baNlh\_G76hEel4Qzo3lKj2soYqFHQHpIC5ENCEcAZI4IeQ1uVJXfh-SGEgDLuHlEMtWe6BOxIllgceb8vk46zN16G6K](https://www.gupy.us/vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGQZiNg6ZP2RNUjze8NqOi5OIR2j0Mu4ZzuUPwqusPKCNFN2B6TGA7ltzYPOuZKRdFq7Jx8N5WY-wi58Z7-ZZsd3yeN-baNlh_G76hEel4Qzo3lKj2soYqFHQHpIC5ENCEcAZI4IeQ1uVJXfh-SGEgDLuHlEMtWe6BOxIllgceb8vk46zN16G6K)
> 4.  **Product Designer Pleno na Afya**
> 
>     *   **Localização:** Híbrido (Rio de Janeiro, São Paulo ou Belo Horizonte)
>     *   **Requisitos:** Conduzir pesquisas com usuários e stakeholders, desenvolver wireframes, protótipos e interfaces de alta fidelidade no Figma, estruturar fluxos de navegação e jornadas do usuário, aplicar princípios de UX Writing, acessibilidade e arquitetura da informação.
>     *   **Link:** [https://www.gupy.us/vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHaI\_0vK7lwFyGE7Mm9S8zgNifn8qxsQWE3EbXfafHGhzISOPtFkTkBfpYWhoui92dFJ87OF4qLwzF6Fb1aqciVxK0NOS8nD3TwYpLh9si5ZA6Se1p7k1iiF30LTSreEUXO7UjRTNmn-yNAWP\_olxXDuMQNBOO3L2CgIN00](https://www.gupy.us/vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHaI_0vK7lwFyGE7Mm9S8zgNifn8qxsQWE3EbXfafHGhzISOPtFkTkBfpYWhoui92dFJ87OF4qLwzF6Fb1aqciVxK0NOS8nD3TwYpLh9si5ZA6Se1p7k1iiF30LTSreEUXO7UjRTNmn-yNAWP_olxXDuMQNBOO3L2CgIN00)
> 
> Espero que estas opções sejam úteis!


--------------------------------------------------------------

🔄 Processando resultados de planejamento...

 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 



> Com certeza! Fico feliz em te ajudar a se destacar nos processos seletivos para as vagas de UX Design de nível pleno.
> 
> **1. Habilidades e experiências a enfatizar:**
> 
> *   **Experiência prática:** Demonstre experiência sólida em projetos de UX, desde a pesquisa inicial até a implementação e testes. Destaque sua participação em todas as etapas do processo de design.
> *   **Domínio de ferramentas:** Certifique-se de ter domínio de ferramentas como Figma, Sketch e Adobe XD. Mencione sua familiaridade com outras ferramentas relevantes para o design de UX.
> *   **Design Centrado no Usuário:** Mostre seu conhecimento e experiência em aplicar os princípios do Design Centrado no Usuário para criar soluções que atendam às necessidades dos usuários.
> *   **Resolução de problemas:** Enfatize sua capacidade de transformar requisitos de negócio em soluções funcionais e intuitivas.
> *   **Colaboração:** Destaque sua capacidade de trabalhar em equipe e colaborar com times multidisciplinares.
> *   **Habilidades de comunicação:** A comunicação clara e eficaz é fundamental para um UX Designer. Demonstre suas habilidades de comunicação ao explicar suas decisões de design e apresentar suas ideias.
> *   **Conhecimento técnico:** Ter conhecimento básico de HTML, CSS e JavaScript pode ser um diferencial. Isso demonstra sua capacidade de entender as limitações e possibilidades técnicas do desenvolvimento web.
> 
> **2. Dicas de organização para se candidatar:**
> 
> *   **Adapte seu currículo:** Ajuste seu currículo para cada vaga, destacando as habilidades e experiências mais relevantes para a posição. Use palavras-chave presentes na descrição da vaga.
> *   **Pesquise a empresa:** Antes da entrevista, pesquise a empresa, seus produtos e serviços. Entenda a cultura da empresa e como suas habilidades podem contribuir para o sucesso da organização.
> 
> **3. Dicas gerais para se destacar no mercado de trabalho de UX Design:**
> 
> *   **Mantenha-se atualizado:** A área de UX Design está em constante evolução. Acompanhe as tendências do mercado, participe de cursos e workshops e leia artigos e livros sobre o assunto.
> *   **Construa um portfólio sólido:** Um portfólio online é essencial para mostrar seus projetos e habilidades. Inclua projetos que demonstrem sua capacidade de resolver problemas de design e criar experiências de usuário agradáveis.
> *   **Networking:** Participe de eventos e grupos de discussão sobre UX Design. Conecte-se com outros profissionais da área para trocar ideias e oportunidades.
> *   **Desenvolva suas habilidades de UX Writing:** A habilidade de escrever textos claros, concisos e informativos é cada vez mais valorizada em UX Design. Pratique a escrita de microtextos, mensagens de erro e outros elementos textuais da interface.
> *   **Acessibilidade:** Demonstre conhecimento e preocupação com a acessibilidade em seus projetos. Certifique-se de que suas soluções de design sejam inclusivas e atendam às necessidades de todos os usuários.
> 
> Espero que estas dicas te ajudem a se preparar e se destacar nos processos seletivos! Se tiver mais alguma dúvida, é só perguntar.
> 


--------------------------------------------------------------
Se deseja procurar outra vaga, digite 1: 

0

 Até a próxima! 😊
